# Homework 1
Diego Rivera Garrido(1674196), d.rivera.garrido@student.tue.nl

Iliyan vasilev Teofilov(1671952), i.v.teofilov@student.tue.nl

Nicolas Martinez van der Looven(2064839), n.martinez.van.der.looven@student.tue.nl

In [ ]:
import numpy as np

## 1 Gradient Descent

First we implemented python functions forthe sigmoide, f and ∇f.

In [ ]:
def sigmoide(x):
    return np.exp(x)/(1+np.exp(x))

def f(x):
    w,b = x[0],x[1]
    #return -np.log(sigmoide(w+b))-np.log(sigmoide(1.5*w+b))-np.log(sigmoide(-2*w-b))
    return -0.5*w -b + np.log(1 + np.exp(w+b)) + np.log(1 + np.exp(1.5*w+b))+ np.log(1 + np.exp(-2*w-b))

def grad_f(x):
    w,b = x[0],x[1]
    return np.array([-0.5 + sigmoide(w+b) + 1.5*sigmoide(1.5*w+b) - 2*sigmoide(-2*w-b),
            -1 + sigmoide(w+b) + sigmoide(1.5*w+b) - sigmoide(-2*w-b)])

Below we implemented the gradient descent algorithm given starting point w₀,b₀, f, ∇f and the stepsize function η(t). The result is the last iteration function value f(x) and the lowest (best) function value achieved throughout all iterations.

In [ ]:
def gradient_descent(f, grad_f, eta, w_0, b_0, max_iter=100):
    x = np.array([w_0,b_0])
    fmin = np.inf
    for t in range(0,max_iter):
        x = x - eta(t)*grad_f(x)
        fx = f(x)
        if (fx < fmin): 
            fmin = fx
    return fx,fmin

#### 1.a
Run algorithm with  constant stepsize strategy, η=0.2 constant.

In [ ]:
def eta_const(t,c=0.2):
    return c

In [ ]:
gradient_descent(f,grad_f,eta_const,1,1,100)

#### 1.b
Run algorithm with decreasing step-size strategy, η(t)=0.2/√(t+1).

In [ ]:
def eta_sqrt(t,c=0.2):
    return c/(np.sqrt(t+1))

In [ ]:
gradient_descent(f,grad_f,eta_sqrt,1,1,100)

#### 1.c
Run algorithm with multi-step step-size strategy.

In [ ]:
def eta_multistep(t, milestones=[20,50,80], c=0.2, eta_init=0.2):
    factor = len(milestones)
    for i in range(len(milestones)):
        if t < milestones[i]:
            factor = i
            break
    return eta_init * c**factor

In [ ]:
gradient_descent(f,grad_f,eta_multistep,1,1,100)

## 2 Coordinate Descent

Below we implemented a function for evaluating f(x).

In [ ]:
def f(x):
    return (1/2) *(x[0] ** 4) - x[0] * x[1] + (x[1] ** 2) + x[1] * x[2] + (x[2] ** 2)

#### 2.a
To compute argmin_xi f(x), we will compute the partial derivative of f wrt xi and equal it to zero to check the stationary points, in other words, we solve for xi in df(x)/dxi=0. Next, we find which stationary point evaluated on the second derivative wrt xi yields a positive value to find the minimum.

For x₁, we solve df/dx₁ = 2x₁^3 - x₂ = 0 and d²f/dx₁² = 6x₁^2 > 0 -> x₁ = (x₂/2)^⅓ 

For x₂, we solve df/dx₂ = -x₁ + 2x₂ + x₃ = 0 and d²f/dx₂² = 2 > 0 -> x₂ = (x₁ - x₃)/2

For x₃, we solve df/dx₃ = x₂ + 2x₃ = 0 and d²f/dx₃² = 2 > 0 -> x₃ = -x₂/2

In [ ]:
def argmin(x): # mind the index shift
    # x₁ = (x₂/2)^⅓ 
    argminx0 = pow(x[1]/2, 1/3).real # truncate to real part
    # x₂ = (x₁ - x₃)/2
    argminx1 = (x[0]-x[2])/2
    # x₃ = -x₂/2
    argminx2 =  -x[1]/2
    return (argminx0,argminx1,argminx2)


In [ ]:
x_0=[2,3,4]
for i in range(0,3):
    print("i = " + str(i) + ": " + str(argmin(x_0)[i]))
    print()


## quick test
for t in range(-100, 100):
    if ( f( [t, x_0[1], x_0[2]] ) < f( [argmin(x_0)[0], x_0[1], x_0[2]] ) ):
        print("not min.")
    if ( f( [x_0[0], t, x_0[2]] ) < f( [x_0[0], argmin(x_0)[1], x_0[2]] ) ):
        print("not min.")
    if ( f( [x_0[0], x_0[1], t] ) < f( [x_0[0], x_0[1], argmin(x_0)[2]] ) ):
        print("not min.")

#### 2.b

Next, we implement the coordinate descent algorithm given f, the array of argmin and the starting point x0.

In [ ]:

def coordinate_descent(f, argmin, x_0, max_iter=100):

    print("t =", 0, ":")
    print("x_t =", x_0)
    print("f(x_t) =", f(x_0))
    print()

    x_t = x_0
    for t in range(1, max_iter+1):
        # use x_t to save values of argmin we find
        print("t =", t, ":")
        for i in range(0,3):
            x_t[i] = argmin(x_t)[i] 
        print("x_t =", x_t)
        print("f(x_t) =", f(x_t))
        print()
    return x_t, f(x_t)


In [ ]:
x_0=[1,20,5]
coordinate_descent(f, argmin, x_0, 100);

## 3 Regression - polynomial features

#### 3.a

In this exercise we will be using the California housing dataset from sklearn.

We will compute the desing matrix for this dataset using a polynomial with degree 2, using the function PolynomialFeatures from sklearn.preprocessing. Because the data given has values in different big ranges, this function cannot compute the values for the matrix properly. To solve this what we do is standardize the data using the function StandardScaler from sklearn.preprocessing.

In [ ]:

import scipy as sp
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

In [ ]:
# Creating the data matrix
california = fetch_california_housing()
D = california.data
y = california.target
n,d = D.shape
print(n,d)


In [ ]:
# Creating a design matrix with polynomial standardized features
aff = PolynomialFeatures(2,include_bias=True)
scaler = StandardScaler()
X = aff.fit_transform(scaler.fit_transform(D))
features = aff.get_feature_names_out(california.feature_names)
print(features)

Next, we will compute the regression model minimizing the RSS for the polynomial
design matrix. To minimize RSS, we solve the system of equations given by ∇(RSS)=0

In [ ]:

beta = sp.linalg.solve(X.T@X,X.T@y)
beta

In [ ]:
# Fetch beta values for given features
print(beta[np.where(features == "MedInc")])
print(beta[np.where(features == "MedInc AveBedrms")])
print(beta[np.where(features == "HouseAge AveBedrms")])

#### 3.b

In this subexercise we will repeat what we did in 3.a, in this case using ridge regression with the following objective: \
min f(β) = min 1/n*‖y-Xβ‖²+λ‖β‖²

In [ ]:
# Minimize Objective function f(β)=1/n*‖y-Xβ‖²+λ‖β‖² by solving the system of equations 
# given by ∇(f(β))=0 for λ=0.1
p = X.shape[1]
beta_b = sp.linalg.solve(X.T@X+n*0.1*np.eye(p),X.T@y)
beta_b

In [ ]:
# Fetch beta values for given features
print(beta_b[np.where(features == "MedInc")])
print(beta_b[np.where(features == "MedInc AveBedrms")])
print(beta_b[np.where(features == "HouseAge AveBedrms")])

## Bias-var trade 

In this exercise we will compute the bias² and variance of the given regression models: 
<ul>
<li> True regression function: f*(x)=tan(πx)
    <li> Fitted three regression models on the i.i.d. data samples D1,D2,D3 obtaining:
​    <ul>
        <li> f₁(x) = x + 0.2
        <li> f₂(x) = 3x + 0.3
        <li> f₃(x) = 5x + 0.1
    <ul>    
<ul>
 

In [ ]:
import matplotlib.pyplot as plt

First we define functions to evaluate f*, f1, f2 and f3

In [ ]:
def fstar(x): return np.tan(np.pi*x)
def fd1(x): return x + 0.2
def fd2(x): return 3*x + 0.3
def fd3(x): return 5*x + 0.1

Lets plot the function to have and idea of how the regression functions fit the true function.

In [ ]:
plt.figure()
x = np.linspace(-0.4,0.4)

plt.plot(x,fstar(x), label='f*')
plt.plot(x,fd1(x), label='f1')
plt.plot(x,fd2(x), label='f2')
plt.plot(x,fd3(x), label='f3')
plt.legend()

Finally, lets compute for x₀=0 the bias² and the variance of the models.

In [ ]:
fdx0 = np.array([fd1(0),fd2(0),fd3(0)])
E = np.mean(fdx0)
bias = (fstar(0)-E)
var = np.mean((E-fdx0)**2)

print(bias**2)
print(var)

## Naive Bayes - 20news
From the 20Newsgroups dataset we fetch the documents belonging to three categories, which we use as classes.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.politics.guns',
              'sci.space']

In [ ]:
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

For example, the first document in the training data is the following one:

In [ ]:
print(train.data[0])

The classes are indicated categorically with indices from zero to two by the target vector. The target names tell us which index belongs to which class.

In [ ]:
y_train = train.target
y_train

In [ ]:
train.target_names

We represent the documents in a bag of word format. That is, we create a data matrix ``D`` such that ``D[j,i]=1`` if the j-th document contains the i-th feature (word), and ``D[j,i]=0`` otherwise. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english", min_df=5,token_pattern="[^\W\d_]+", binary=True)
D = vectorizer.fit_transform(train.data)
D_test = vectorizer.transform(test.data)

We get the allocation of feature indices to words by the following array, containing the vocabulary.


In [ ]:
vectorizer.get_feature_names_out()

For example, the word `naive` has the index 4044.

In [ ]:
np.where(vectorizer.get_feature_names_out() == 'naive')[0]

#### 5.a
First lets compute the class prior probabilities p(y):


In [ ]:
y_train_0 = np.array([x for x in y_train if x == 0])
y_train_1 = np.array([x for x in y_train if x == 1])
y_train_2 = np.array([x for x in y_train if x == 2])

p_train_0 = y_train_0.size / y_train.size
p_train_1 = y_train_1.size / y_train.size
p_train_2 = y_train_2.size / y_train.size

p_train_0, p_train_1, p_train_2

#### 5.b
Lets continue by computing the log-probabilities of the word 'naive' given each class, using Laplace smoothing with α=1e-5

In [ ]:
alpha = 1e-5
# Count number of documents of each class in the training set
I_0 = np.where(y_train == 0)[0]
I_1 = np.where(y_train == 1)[0]
I_2 = np.where(y_train == 2)[0]

# Count number of documents per class which includes the word 'naive'
class_counts = {0: 0, 1: 0, 2: 0}
for i in range(y_train.size):
    if D[i, 4044] == 1:
        class_counts[y_train[i]] += 1

K = vectorizer.get_feature_names_out().size

p_train_0 = (class_counts[0] + alpha) / (I_0.size + alpha * K)
p_train_1 = (class_counts[1] + alpha) / (I_1.size + alpha * K)
p_train_2 = (class_counts[2] + alpha) / (I_2.size + alpha * K)

np.log(np.array([p_train_0, p_train_1, p_train_2]))

#### 5.c
Finally, we will compute the class-conditioned log-probabilities for each word and class combination for the first document in the training dataset, using the naive Bayes algorithm.

In [ ]:
alpha = 1e-5
# Count number of documents of each class in the training set
I_0 = np.where(y_train == 0)[0]
I_1 = np.where(y_train == 1)[0]
I_2 = np.where(y_train == 2)[0]

# log p(y=c)
y_train_0 = np.array([x for x in y_train if x == 0])
y_train_1 = np.array([x for x in y_train if x == 1])
y_train_2 = np.array([x for x in y_train if x == 2])

log_p_train_0 = np.log(y_train_0.size / y_train.size)
log_p_train_1 = np.log(y_train_1.size / y_train.size)
log_p_train_2 = np.log(y_train_2.size / y_train.size)

# p(xd = xdtest | y = c)
counts = {}
num_words = vectorizer.get_feature_names_out().size
train_size = y_train.size
for w in range(num_words):
    counts[w, 0] = 0
    counts[w, 1] = 0
    counts[w, 2] = 0
for w in range(num_words):
    for i in range(train_size):
        if D[i,w] == 1:
            counts[w, y_train[i]] += 1

# log p(y=c)+sum(log(p(xd = xdtest | y = c)))
for w in range(num_words):
    if D[0, w] == 1:
        log_p_train_0 += np.log((counts[w, 0] + alpha) / (I_0.size + alpha * num_words))
        log_p_train_1 += np.log((counts[w, 1] + alpha) / (I_1.size + alpha * num_words))
        log_p_train_2 += np.log((counts[w, 2] + alpha) / (I_2.size + alpha * num_words))

log_p_train_0, log_p_train_1, log_p_train_2